In [71]:
import pandas as pd

df = pd.read_csv("data/train.csv")

other = pd.read_json("data/id_mappings.json")

# Extract the dictionary inside "user_mapping"
user_mapping = other["user_mapping"]
item_mapping = other["item_mapping"]

# Convert to DataFrame
user_mapping_df = pd.DataFrame(list(user_mapping.items()), columns=["user_id", "user_index"])
item_mapping_df = pd.DataFrame(list(item_mapping.items()), columns=["item_str_code", "item_id"])

item_metadata_df["item_id"] = item_metadata_df["parent_asin"].map(item_mapping)
item_merge_df = pd.merge(item_metadata_df, item_mapping_df, how="left", left_on="parent_asin", right_on="item_str_code")
item_merge_df["item_id"] = item_merge_df["item_id_x"]



item_merge_df["item_id"] = item_merge_df["item_id"].astype(float)
item_merge_df = item_merge_df.drop('item_id_x', axis=1)
item_merge_df = item_merge_df.drop('item_id_y', axis=1)
item_merge_df = item_merge_df.drop('item_str_code', axis=1)


result = pd.merge(df, item_merge_df, how="left", left_on="item_id", right_on="item_id")
result

,user_id,item_id,rating,timestamp,parent_asin,main_category,title,average_rating,rating_number,price,store,features,description,images,categories,image_count,has_images,image_urls,category
0,0,7314,5.0,1353612262000,B006CQ8TC2,Appstore for Android,Turkey Season,3.5,304.0,0.99,"Gibbed Games, LLC","['Hunt and slay deviant turkeys, then go back ...",['Gobble gobble gobble! Turkeys are running am...,"{'hi_res': array([None, None, None, None, None...",[],6,True,['https://m.media-amazon.com/images/I/71oAt4H2...,Software
1,0,15493,5.0,1370653034000,B00B7S5FDG,Appstore for Android,Another Monster at the End of This Book...Star...,4.3,460.0,3.99,Sesame Workshop,['Bursting with innovative activities woven ri...,"[""Available for Kindle Fire tablets AND Fire T...","{'hi_res': array([None, None, None, None, None...",[],12,True,['https://m.media-amazon.com/images/I/B1dGfKbS...,Software
2,0,18817,4.0,1373668644000,B00DRPVE3M,Appstore for Android,UNO ™ & Friends - The Classic Card Game Goes S...,3.9,11842.0,NaN,Gameloft,['Create games and invite your friends to play...,"['UNO™ & Friends: Fast Fun for Everyone!UNO™, ...","{'hi_res': array([None, None, None, None, None...",[],6,True,['https://m.media-amazon.com/images/I/A1dc9cyK...,Software
3,0,18251,4.0,1373669469000,B00D948SS0,Appstore for Android,World's Biggest Wordsearch - Your daily free w...,3.8,8290.0,0.00,Supersonic Software Ltd,"['5,000 words to find' '361 puzzles based on a...","[""Find thousands of words in the biggest and b...","{'hi_res': array([None, None, None, None, None...",[],6,True,['https://m.media-amazon.com/images/I/A1cI1uy6...,Software
4,0,16199,1.0,1375136041000,B00BQPEIT2,Appstore for Android,Temple Run: Oz,4.0,1043.0,NaN,Disney,['Stunning environments inspired by the film –...,['The most thrilling running experience now co...,"{'hi_res': array([None, None, None, None, None...",[],6,True,['https://m.media-amazon.com/images/I/91BmBng4...,Software
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543141,868216,15203,5.0,1397341480000,B00B1TGUMG,Software,Microsoft Office Home and Student 2013 (1PC/1U...,4.3,1759.0,NaN,Microsoft,['NOTE: Windows XP and Vista NOT supported. Th...,['Product Description'\n 'Office Home & Studen...,{'hi_res': array(['https://m.media-amazon.com/...,['Software' 'Microsoft' 'All Microsoft'],6,True,['https://m.media-amazon.com/images/I/61vi4XLD...,Software
2543142,868216,51839,5.0,1500036628141,B06XS4YRWX,Software,"NTI Echo 3, Make an exact copy of a HDD or SSD...",3.5,17.0,NaN,Nti,"['Versatile: One-to-One Cloning, Dynamic Resiz...","['NTI Echo'\n ""is the perfect solution for all...","{'hi_res': array([None, None, None, None, None...",['Software' 'Utilities' 'Backup'],14,True,['https://m.media-amazon.com/images/I/81FlCZVs...,Software
2543143,868217,14052,4.0,1464923235000,B00AB7HESI,Appstore for Android,Scribblenauts Remix,3.8,5520.0,0.99,Warner Bros,['CLASSIC GAMEPLAY - 50 levels to play!'\n 'PL...,['THINK IT! CREATE IT! SOLVE IT! The award-win...,"{'hi_res': array([None, None, None, None, None...",[],6,True,['https://m.media-amazon.com/images/I/81bAX50X...,Software
2543144,868217,40099,4.0,1465146950000,B014RGFC0K,Appstore for Android,Smashy Road: Wanted,3.6,7541.0,0.00,Bearbit Studios B.V.,['90 unlockable vehicles!!'\n 'Random generate...,['You are WANTED! Take your driving to new hei...,"{'hi_res': array([None, None, None, None, None...",[],9,True,['https://m.media-amazon.com/images/I/71TPOiUY...,Software
